# Project Messaging

In [ ]:
import syft as sy
sy.requires(">=0.8.1-beta")
sy.enable_autoreload()
from syft.service.project.project import ProjectMessage, ProjectThreadMessage

In [ ]:
alice_domain = sy.orchestra.launch(name="Alice", dev_mode=True)
alice_domain_client = alice_domain.login(email="info@openmined.org", password="changethis")

In [ ]:
bob_domain = sy.orchestra.launch(name="Bob", dev_mode=True)
bob_domain_client = bob_domain.login(email="info@openmined.org", password="changethis")

## Retrieving existing projects

In [ ]:
alice_projects = alice_domain_client.api.services.project.get_all()
alice_project = alice_projects[0]
assert len(alice_projects)==1
alice_projects

In [ ]:
bob_projects = bob_domain_client.api.services.project.get_all()
bob_project = bob_projects[0]
assert len(bob_projects)==1
bob_projects

In [ ]:
assert alice_project.user_signing_key == alice_domain_client.api.signing_key

In [ ]:
assert bob_project.user_signing_key == bob_domain_client.api.signing_key

In [ ]:
assert len(alice_project.get_messages()) == 0
alice_project.messages

In [ ]:
assert len(bob_project.get_messages()) == 0
bob_project.messages

## 1. Initial message by Alice

In [ ]:
alice_project.send_message(message="Hello Everyone 👋, I am Alice")

In [ ]:
assert len(alice_project.get_messages()) == 1
print(alice_project.messages)

In [ ]:
assert len(bob_project.get_messages()) == 0
bob_project.messages

## 2. Project Sync Functionality

In [ ]:
bob_project.sync()

In [ ]:
assert len(bob_project.get_messages()) == 1
print(bob_project.messages)

In [ ]:
bob_project.send_message(message="Hello Everyone 👋, I am Bob")

In [ ]:
assert len(bob_project.get_messages()) == 2
print(bob_project.messages)

In [ ]:
assert len(alice_project.get_messages()) == 1
print(alice_project.messages)

## 3. Project Auto Sync Functionality

In [ ]:
#alice sending message without sync, which inturn does auto sync at backend
alice_project.send_message(message="Glad to working on this project with you all")

In [ ]:
assert len(alice_project.get_messages()) == 3
print(alice_project.messages)

## 4. Project MessageThread Reply

In [ ]:
# to check sub thread reply by bob
bob_project.sync()
assert len(bob_project.get_messages()) == 3
print(bob_project.messages)

In [ ]:
bob_project.reply_message(reply = "Likewise 🤘", message=bob_project.events[2])

In [ ]:
assert len(bob_project.get_messages()) == 4
print(bob_project.messages)

In [ ]:
assert bob_project.events[3].parent_event_id == bob_project.events[2].id

## 5. Adding messages to existing sub threads

In [ ]:
#sub threaded reply to existing thread
alice_project.sync()
alice_project.reply_message(reply = "Go Team ❤️!", message=alice_project.events[3].id)

In [ ]:
assert len(alice_project.get_messages()) == 5
print(alice_project.messages)

In [ ]:
assert alice_project.events[4].parent_event_id == bob_project.events[2].id

In [ ]:
assert len(alice_project.events)==5

In [ ]:
assert len(alice_project.event_id_hashmap)==5

In [ ]:
res =  alice_project.validate_events(debug=True)
res

In [ ]:
assert isinstance(res, sy.SyftSuccess)

In [ ]:
res =  bob_project.validate_events(debug=True)
res

In [ ]:
assert isinstance(res, sy.SyftSuccess)

In [ ]:
for node in [alice_domain, bob_domain]:
    if node.node_type.value == "python":
        node.land()